# Layout barrier position-family comparison

This standalone notebook reproduces the first comparison in the paper: the position-family study at the layout barrier.

Upload **this notebook only** to Google Colab, enable a GPU, and run all cells. No extra `.py` files or archives are required.

Compared families:

- **abspos_full2**: learned absolute-position baseline
- **mixedlayout_abspos**: absolute-position control with mixed 2-digit / 3-digit layout exposure
- **pcsig_full2**: coupled-significance position IDs
- **digaware_full2**: digit-aware / abacus-like significance IDs layered on absolute positions
- **symmcol_full2**: symmetry-aware column IDs
- **nope_full2**: no explicit position signal

Primary question:

Can positional inductive bias alone remove the earliest exact-match failure, or is direct exposure to the shifted layout necessary?


In [ ]:

from __future__ import annotations

# ====== Basic configuration ======
RUN_PROFILE = "layout_barrier_position_family_comparison"
OUTPUT_ROOT = "results_layout_barrier_position_families"
AUTO_DOWNLOAD_ZIP = True
FORCE_CLEAN_OUTPUT_ROOT = True
SAVE_MODEL_STATE = False

# Default hyperparameters
N_LAYER = 1
N_EMBD = 16
N_HEAD = 4
NUM_STEPS = 5000
BATCH_SIZE = 256
LEARNING_RATE = 3e-3
WEIGHT_DECAY = 0.0
EVAL_N_PER_SUITE = 1000
BASE_SEEDS = [0, 1, 2, 3, 4]
INCLUDE_NOPE = True

print({
    "RUN_PROFILE": RUN_PROFILE,
    "OUTPUT_ROOT": OUTPUT_ROOT,
    "SAVE_MODEL_STATE": SAVE_MODEL_STATE,
    "N_LAYER": N_LAYER,
    "N_EMBD": N_EMBD,
    "N_HEAD": N_HEAD,
    "NUM_STEPS": NUM_STEPS,
    "BATCH_SIZE": BATCH_SIZE,
    "LEARNING_RATE": LEARNING_RATE,
    "WEIGHT_DECAY": WEIGHT_DECAY,
    "EVAL_N_PER_SUITE": EVAL_N_PER_SUITE,
    "BASE_SEEDS": BASE_SEEDS,
    "INCLUDE_NOPE": INCLUDE_NOPE,
})

import json
import math
import os
import shutil
import zipfile
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import torch

# ====== Embedded common module (standalone) ======
import base64
import sys
import types

_EMBEDDED_COMMON_B64 = """..."""

if "microgpt_tmlr_extension_common" not in sys.modules:
    _embedded_common = types.ModuleType("microgpt_tmlr_extension_common")
    exec(
        compile(
            base64.b64decode(_EMBEDDED_COMMON_B64).decode("utf-8"),
            "microgpt_tmlr_extension_common.py",
            "exec",
        ),
        _embedded_common.__dict__,
    )
    sys.modules["microgpt_tmlr_extension_common"] = _embedded_common

print("embedded module loaded: microgpt_tmlr_extension_common")

from microgpt_tmlr_extension_common import (
    ABS_POSITION_ROLE_ALIASING,
    COUPLED_POSITION_SCHEME,
    SIG_ONLY_SCHEME,
    SYMM_ROLE_SCHEME,
    build_full_train_docs_2digit,
    build_full_train_docs_3digit_lowrange,
    build_eval_rows_from_pairs,
    required_block_size,
    sample_pairs_2digit,
    sample_pairs_true_3digit_conditioned,
    train_one_experiment,
)


device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)
if device == "cuda":
    print("gpu:", torch.cuda.get_device_name(0))


def build_design_manifest() -> pd.DataFrame:
    rows = [
        {
            "family": "abspos_full2",
            "train_family": "full2",
            "model_kind": "abspos",
            "family_group": "absolute_positions",
            "literature_inspiration": "standard learned absolute positions",
        },
        {
            "family": "mixedlayout_abspos",
            "train_family": "mixedlayout",
            "model_kind": "abspos",
            "family_group": "absolute_positions_plus_layout_exposure",
            "literature_inspiration": "layout-exposed absolute position control",
        },
        {
            "family": "pcsig_full2",
            "train_family": "full2",
            "model_kind": "coupled_sig",
            "family_group": "position_coupling_type",
            "literature_inspiration": "minimal position-coupling-style significance IDs",
        },
        {
            "family": "digaware_full2",
            "train_family": "full2",
            "model_kind": "absplussig",
            "family_group": "digit_aware_type",
            "literature_inspiration": "absolute positions + local digit-significance IDs",
        },
        {
            "family": "symmcol_full2",
            "train_family": "full2",
            "model_kind": "symmcol",
            "family_group": "symmetry_aware_type",
            "literature_inspiration": "column-shared + operand-symmetric role IDs",
        },
    ]
    if INCLUDE_NOPE:
        rows.append({
            "family": "nope_full2",
            "train_family": "full2",
            "model_kind": "nope",
            "family_group": "no_position_baseline",
            "literature_inspiration": "NoPE context baseline",
        })
    return pd.DataFrame(rows)


DESIGN_MANIFEST = build_design_manifest()
print(DESIGN_MANIFEST)


def build_train_docs(train_family: str):
    if train_family == "full2":
        return build_full_train_docs_2digit()
    if train_family == "mixedlayout":
        return build_full_train_docs_2digit() + build_full_train_docs_3digit_lowrange()
    raise ValueError(train_family)


def build_experiments():
    experiments = []
    block_size = max(required_block_size(2), required_block_size(3))  # 13
    for row in DESIGN_MANIFEST.to_dict(orient="records"):
        train_docs = build_train_docs(row["train_family"])
        for seed in BASE_SEEDS:
            eval_rows = []
            eval_rows.extend(build_eval_rows_from_pairs(
                sample_pairs_2digit(EVAL_N_PER_SUITE, seed=1000 + seed),
                width_in=2, width_out=3, suite="in_support_2digit",
            ))
            eval_rows.extend(build_eval_rows_from_pairs(
                sample_pairs_2digit(EVAL_N_PER_SUITE, seed=2000 + seed),
                width_in=3, width_out=4, suite="layout_shift_only_lowrange3",
            ))
            eval_rows.extend(build_eval_rows_from_pairs(
                sample_pairs_true_3digit_conditioned(
                    EVAL_N_PER_SUITE,
                    seed=3000 + seed,
                    incoming_carry_to_hundreds=0,
                    carryout_from_hundreds=0,
                ),
                width_in=3, width_out=4, suite="true3_noincarry",
            ))
            eval_rows.extend(build_eval_rows_from_pairs(
                sample_pairs_true_3digit_conditioned(
                    EVAL_N_PER_SUITE,
                    seed=4000 + seed,
                    incoming_carry_to_hundreds=1,
                    carryout_from_hundreds=0,
                ),
                width_in=3, width_out=4, suite="true3_withincarry",
            ))
            eval_rows.extend(build_eval_rows_from_pairs(
                sample_pairs_true_3digit_conditioned(
                    EVAL_N_PER_SUITE,
                    seed=5000 + seed,
                    carryout_from_hundreds=1,
                ),
                width_in=3, width_out=4, suite="true3_thousands_carry",
            ))
            experiments.append({
                "name": f"{row['family']}_seed{seed}",
                "family": row["family"],
                "train_family": row["train_family"],
                "model_kind": row["model_kind"],
                "family_group": row["family_group"],
                "literature_inspiration": row["literature_inspiration"],
                "seed": seed,
                "block_size": block_size,
                "num_steps": NUM_STEPS,
                "batch_size": BATCH_SIZE,
                "learning_rate": LEARNING_RATE,
                "weight_decay": WEIGHT_DECAY,
                "n_layer": N_LAYER,
                "n_embd": N_EMBD,
                "n_head": N_HEAD,
                "train_docs": train_docs,
                "eval_rows": eval_rows,
                "n_train_docs": len(train_docs),
                "n_eval": len(eval_rows),
            })
    return experiments


EXPERIMENTS = build_experiments()
print(pd.DataFrame([{
    "name": e["name"],
    "family": e["family"],
    "train_family": e["train_family"],
    "model_kind": e["model_kind"],
    "seed": e["seed"],
    "block_size": e["block_size"],
    "num_steps": e["num_steps"],
    "n_train_docs": e["n_train_docs"],
    "n_eval": e["n_eval"],
} for e in EXPERIMENTS]))


output_root = Path(OUTPUT_ROOT)
if FORCE_CLEAN_OUTPUT_ROOT and output_root.exists():
    shutil.rmtree(output_root)
output_root.mkdir(parents=True, exist_ok=True)

ABS_POSITION_ROLE_ALIASING.to_csv(output_root / "absolute_position_role_aliasing.csv", index=False)
COUPLED_POSITION_SCHEME.to_csv(output_root / "coupled_position_scheme.csv", index=False)
SIG_ONLY_SCHEME.to_csv(output_root / "digit_significance_scheme.csv", index=False)
SYMM_ROLE_SCHEME.to_csv(output_root / "symmetry_role_scheme.csv", index=False)
DESIGN_MANIFEST.to_csv(output_root / "design_manifest.csv", index=False)

suite_rows = []
pred_paths = []
manifest_rows = []

for exp in EXPERIMENTS:
    workdir = output_root / exp["name"]
    result = train_one_experiment(
        name=exp["name"],
        family=exp["family"],
        train_family=exp["train_family"],
        model_kind=exp["model_kind"],
        seed=exp["seed"],
        block_size=exp["block_size"],
        num_steps=exp["num_steps"],
        batch_size=exp["batch_size"],
        learning_rate=exp["learning_rate"],
        weight_decay=exp["weight_decay"],
        n_layer=exp["n_layer"],
        n_embd=exp["n_embd"],
        n_head=exp["n_head"],
        train_docs=exp["train_docs"],
        eval_rows=exp["eval_rows"],
        device=device,
        save_model_state=SAVE_MODEL_STATE,
        workdir=workdir,
    )
    pred_df = result["pred_df"].copy()
    pred_df.insert(0, "seed", exp["seed"])
    pred_df.insert(0, "model_kind", exp["model_kind"])
    pred_df.insert(0, "train_family", exp["train_family"])
    pred_df.insert(0, "family", exp["family"])
    pred_df.to_csv(workdir / "predictions_full.csv", index=False)

    suite_rows.append(result["suite_df"])
    manifest_rows.append({
        "name": exp["name"],
        "family": exp["family"],
        "train_family": exp["train_family"],
        "model_kind": exp["model_kind"],
        "family_group": exp["family_group"],
        "literature_inspiration": exp["literature_inspiration"],
        "seed": exp["seed"],
        "block_size": exp["block_size"],
        "num_steps": exp["num_steps"],
        "batch_size": exp["batch_size"],
        "learning_rate": exp["learning_rate"],
        "weight_decay": exp["weight_decay"],
        "n_layer": exp["n_layer"],
        "n_embd": exp["n_embd"],
        "n_head": exp["n_head"],
        "n_train_docs": exp["n_train_docs"],
        "n_eval": exp["n_eval"],
    })

manifest_df = pd.DataFrame(manifest_rows)
manifest_df.to_csv(output_root / "manifest.csv", index=False)

suite_df = pd.concat(suite_rows, ignore_index=True)
suite_df.to_csv(output_root / "suite_summary.csv", index=False)

suite_mean_summary = suite_df.groupby(
    ["family", "train_family", "model_kind", "suite"]
).agg(
    seeds=("seed", "nunique"),
    exact_constrained_mean=("exact_constrained", "mean"),
    exact_constrained_std=("exact_constrained", "std"),
    exact_constrained_min=("exact_constrained", "min"),
    exact_constrained_max=("exact_constrained", "max"),
    teacher_token_accuracy_mean=("teacher_token_accuracy", "mean"),
    teacher_token_accuracy_std=("teacher_token_accuracy", "std"),
).reset_index()
suite_mean_summary.to_csv(output_root / "suite_mean_summary.csv", index=False)

journal_target_summary = suite_mean_summary[
    suite_mean_summary["suite"].isin([
        "layout_shift_only_lowrange3",
        "true3_noincarry",
        "true3_withincarry",
        "true3_thousands_carry",
    ])
].copy()
journal_target_summary.to_csv(output_root / "journal_target_summary.csv", index=False)

# Pairwise delta against abspos_full2 for the four OOD suites
baseline = journal_target_summary[journal_target_summary["family"] == "abspos_full2"][
    ["suite", "exact_constrained_mean"]
].rename(columns={"exact_constrained_mean": "abspos_full2_mean"})
pairwise = journal_target_summary.merge(baseline, on="suite", how="left")
pairwise["delta_vs_abspos_full2"] = pairwise["exact_constrained_mean"] - pairwise["abspos_full2_mean"]
pairwise.to_csv(output_root / "pairwise_delta_vs_abspos_full2.csv", index=False)

# Make a compact figure for exact constrained means
plot_df = journal_target_summary.copy()
family_order = [f for f in ["abspos_full2", "pcsig_full2", "digaware_full2", "symmcol_full2", "mixedlayout_abspos", "nope_full2"] if f in plot_df["family"].unique()]
suite_order = ["layout_shift_only_lowrange3", "true3_noincarry", "true3_withincarry", "true3_thousands_carry"]

fig, ax = plt.subplots(figsize=(12, 4.2))
x_positions = list(range(len(suite_order)))
bar_width = 0.13 if len(family_order) >= 5 else 0.18

for i, fam in enumerate(family_order):
    sub = plot_df[plot_df["family"] == fam].set_index("suite").reindex(suite_order)
    xs = [x + (i - (len(family_order) - 1) / 2) * bar_width for x in x_positions]
    ax.bar(xs, sub["exact_constrained_mean"].values, width=bar_width, label=fam)
    ax.errorbar(xs, sub["exact_constrained_mean"].values, yerr=sub["exact_constrained_std"].fillna(0).values, fmt="none", capsize=2)

ax.set_xticks(x_positions)
ax.set_xticklabels(suite_order, rotation=15, ha="right")
ax.set_ylim(0, 1.05)
ax.set_ylabel("Exact match (constrained)")
ax.set_title("Stage16 positional-family comparison")
ax.legend(loc="upper center", bbox_to_anchor=(0.5, 1.30), ncol=3, frameon=True)
fig.tight_layout()
fig.savefig(output_root / "position_family_exact.png", dpi=200, bbox_inches="tight")
plt.close(fig)

readme = """\
Results pack contents (Stage16 positional-family comparison):
- absolute_position_role_aliasing.csv
- coupled_position_scheme.csv
- digit_significance_scheme.csv
- symmetry_role_scheme.csv
- design_manifest.csv
- manifest.csv
- suite_summary.csv
- suite_mean_summary.csv
- journal_target_summary.csv
- pairwise_delta_vs_abspos_full2.csv
- position_family_exact.png
- per-experiment folders with config / train_log / predictions / checkpoint

Interpretation:
- pcsig_full2: position-coupling-type significance IDs
- digaware_full2: absolute positions + local digit-significance IDs
- symmcol_full2: column-shared, operand-symmetric role IDs
- mixedlayout_abspos: layout-exposed absolute-position control

Important note:
The three task-structured families are intended as minimal instantiations inspired by the literature,
not exact reproductions of the full published architectures or data formats.
"""
(output_root / "README_results.txt").write_text(readme, encoding="utf-8")

zip_path = output_root.with_suffix(".zip")
if zip_path.exists():
    zip_path.unlink()
with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as zf:
    for path in output_root.rglob("*"):
        zf.write(path, path.as_posix())

print("done")
print("output_root:", output_root.resolve())
print("zip:", zip_path.resolve())
print(pd.read_csv(output_root / "suite_mean_summary.csv"))

if AUTO_DOWNLOAD_ZIP:
    try:
        from google.colab import files
        files.download(str(zip_path))
    except Exception as e:
        print("auto-download skipped:", repr(e))
